# Classical Mechanics I (PHYS 311)
## Studio 10

*Name:* Connor Haskell

*Date:* 11/1/2024

## The General 3-Body Problem

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


As in the last two weeks, we have these magic lines to give us access to the animation and conversion functionalities.

In [2]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

Last time, we defined a function `plotSomeStuff` that took arrays of x_arrays and arrays of y_arrays allowing us to plot the paths of multiple particles. It should have looked something like this:


In [3]:
def plotSomeStuff(x_arrays, y_arrays, nframes=100, ax = None, fig = None):
    if ax == None or fig == None:
        fig, ax = plt.subplots()
        ax.set_xlim(-10,10)
        ax.set_ylim(-10, 10)

        
    marker = {}
    orbit = {}
    
    for ipath,(x_array, y_array) in enumerate(zip(x_arrays,y_arrays) ):
        # This is an example of how to loop over all pairs of x and y arrays.
        # The zip function takes two lists and combines them into a single list like:
        # zip( [a, b], [c, d] ) will give [ [a,c], [b,d] ]
        # The enumerate function gives you an index for each object in an iterable list.
        # So enumerate([a,b]) will give a new list [ [0,a], [1,b] ]
        marker[ipath], = ax.plot([],"o")
        orbit[ipath], = ax.plot([])

    def animate(frame_num):
        """
        """
        # Now write some code here that would loop over all pairs of the x and y arrays (see above)
        # and then use the set_data function like last week to edit each object in marker and orbit.
        for ipath,(x_array, y_array) in enumerate(zip(x_arrays,y_arrays) ):
            x = x_array[frame_num]
            y = y_array[frame_num]
            marker[ipath].set_data([x,y])
            orbit[ipath].set_data([ x_array[:frame_num],y_array[:frame_num] ])
        return

    anim = FuncAnimation(fig, animate, frames=nframes, interval=20)
    display(HTML(anim.to_jshtml()))
    plt.close()




So now let's use this to help us plot the solutions to the 3-body problem.

Remember that for the 3-body problem, we don't have a handy trick for mapping it onto the motion of a single particle of reduced mass. So that's a bit unfortunate. This now means that we'll need a more complex system of ODEs to solve this. But luckily that's not so hard for the computer to do numerically!

Let's start by importing the odeint function:

In [4]:
from scipy.integrate import odeint

Then we'll need to create a new ODE system. Before we were solving two second order ODEs that describe the motion of one particle in 2D:

$$m\ddot{x}=\sum F_x$$

$$m\ddot{y}=\sum F_y$$

Since odeint only know's how to solve first order ODEs, we expressed this as a system of **four first order ODEs**. And this is what's needed for each particle (but because of the reduced mass trick, it meant last week, we could hand it just four).


$$m\dot{v}_x=\sum F_x$$

$$m\dot{v}_y=\sum F_y$$

$$\dot{x} = v_x$$

$$\dot{y} = v_y$$

Now, we'll need to write an ODE system function that can handle an arbitrary number of particles. I've written it for you, but do make sure you understand what's happening in this `ode_system` function. Then I'll give an example of solving the 2-body problem in this way (i.e. not using the reduced mass trick!).

In [5]:
def ode_system(inputs,t,m):
    """
    This function represents a series of first order ODEs.
    
    Return: List of expressions for the first time derivative of the inputs, in order.
    """
    
    # Parse the inputs list to positions x,y and vector magnitudes xdot,ydot
    # But this time, let's store them in lists since there will be N
    # values of each of these for N particles.
    
    x = []
    y = []
    xdot = []
    ydot = []
    
    Fx = []
    Fy = []
    
    # Let's parse the inputs
    # This expects that the inputs are ordered as:
    # [x1,y1,xdot1,ydot1,x2,y2,xdot2,ydot2,...]
    for inputIndex in range(0,len(inputs),4):
        x.append(inputs[inputIndex])
        y.append(inputs[inputIndex+1])
        xdot.append(inputs[inputIndex+2])
        ydot.append(inputs[inputIndex+3])
    
    # Now the hard part is figuring out the sum of forces for each of these.
    # It's no longer going to be a simple xdot=Fx/m
    # We now need to sum together the gravitational forces of all of the other
    # particles.
    
    # Let's assume the force on particle i from particle j is
    # F_ij = -(mi*mj)/rij^2
    
    # So this loop is over each particle i 
    for iparticle in range(len(x)):
        # The forces on particle i in the x and y directions.
        # Let's initialize them as zero and then add up the components
        Fix = 0
        Fiy = 0
        
        # Position of i:
        xi = x[iparticle]
        yi = y[iparticle]
        
        # We also need the mass of the particles... Let's assume they were 
        # stored in a list in the m object.
        mi = m[iparticle]
        
        for jparticle in range(len(x)):
            # in this double loop, we want to skip when iparticle and jparticle
            # are the same particle. we're saying there is no gravitational self-interaction
            if iparticle==jparticle:
                continue
            # Position of j
            xj = x[jparticle]
            yj = y[jparticle]

            mj = m[jparticle]
            
            # Now we have all the info to calculate the distance between i and j
            rij = sqrt((xi-xj)**2 + (yi-yj)**2)
            
            
            # The magnitude of this force will be
            # -(m1*m2)/rij^2
            # But... we need Fx and Fy. So that's then some trig. We did some
            # of this last time:
            phi = np.arctan2(yi-yj,xi-xj)
            Fijr = -mi*mj/(rij*rij)
            Fijx = Fijr*np.cos(phi)
            Fijy = Fijr*np.sin(phi)
            
            # and add them to our running Fix and Fiy sums.
            # (Google the += operator in python if you haven't seen this before)
            Fix += Fijx
            Fiy += Fijy

        # Now we have all of the forces that act on particle i
        # Let's store them in a list
        Fx.append(Fix)
        Fy.append(Fiy)
    
    # Let's make a list of values to return
    # this will have to be ordered like:
    # [xdot1, ydot1, F1x/m1, F1y/m1, xdot2, ydot2, F2x/m2, F2y/m2, ...]
    
    returnlist = []
    for iparticle in range(len(x)):
        returnlist.append(xdot[iparticle])
        returnlist.append(ydot[iparticle])
        returnlist.append(Fx[iparticle]/m[iparticle])
        returnlist.append(Fy[iparticle]/m[iparticle])

    return returnlist



nframes = 50
tmax = 100
t_array = linspace(0,tmax,nframes)

mlist = [1,1]

solutions = odeint(ode_system, (-2,0,0,-0.4,2,0,0,0.4), t_array, args=(mlist,))

x1_array = solutions[:,0]
y1_array = solutions[:,1]

x2_array = solutions[:,4]
y2_array = solutions[:,5]


plotSomeStuff([x1_array,x2_array],[y1_array,y2_array],nframes)

**In a bulleted list, qualitatively explain what `ode_system` is doing, walking through the logic.** 

Odeint does not solve 2nd order derivtaives, but can do 1st order so we use 4 1st order derivatives relating the position velocities and position forces for x and y positions.

Then it makes a list to place all the inputs into a order

It also uses ranges to store the x-y position values for each particle and the distance between the particles.

In these range functions it also uses the positions to calulate the the forces in the x and y directions acting on both particles.

Then the odesystem takes the inputs and solves for the derivatives with respect to time

When we graph the solutions, It gives a us a good represntaion of two particles moving with gravitational forces between each other

So now... implement a 3-body system in this 2D space and plot the motion of these three particles (**You don't need to reimplement `ode_system`! The one that's already defined is flexible enough to do this for you. So you just need to hand new parameters to `odeint`**). When you add the third particle, give it a tiny mass relative to the above system. (Let's make it less than one percent the mass of anything else.) **(Comment) How does it affect the orbits from the 2-body problem above?**

In [6]:
nframes = 50
tmax = 100
t_array = linspace(0,tmax,nframes)

mlist = [1,1,0.01]


solutions = odeint(ode_system, (-2,0,0,-0.4,2,0,0,0.4,3,0,0,-0.01), t_array, args=(mlist,))

x1_array = solutions[:,0]
y1_array = solutions[:,1]

x2_array = solutions[:,4]
y2_array = solutions[:,5]

x3_array = solutions[:,8]
y3_array = solutions[:,9]


plotSomeStuff([x1_array,x2_array,x3_array],[y1_array,y2_array,y3_array], nframes)

Now try increasing the mass of this third particle and **comment on what happens to the orbits.** Play around with several configurations of initial conditions and masses to find a few qualitatively different evolutions of this system. Think about the sun-earth-moon system for inspiration if you're having trouble.

In [7]:
nframes = 50
tmax = 100
t_array = linspace(0,tmax,nframes)

mlist = [1,1,1]


solutions = odeint(ode_system, (-2,0,0,-0.4,2,0,0,0.4,3,0,0,-0.01), t_array, args=(mlist,))

x1_array = solutions[:,0]
y1_array = solutions[:,1]

x2_array = solutions[:,4]
y2_array = solutions[:,5]

x3_array = solutions[:,8]
y3_array = solutions[:,9]


plotSomeStuff([x1_array,x2_array,x3_array],[y1_array,y2_array,y3_array], nframes)

Comment: When the third particle increases in mass, Its gravitational force increases. So its attraction to other paticles is increased as well. Depending on many initial conditions, the particles can have many different paths. Since the almost massless particle has way more mass now, things become way more complicated as the partilces seem to jumble up closer to each other. Sometimes one particle will switch to being more attracted to another particle and sometimes a particle will shoot out and the other particles will shoot out in a different direction.

Try to find a configuration where one of the particles is ejected and the other two seem happy as a stable 2-body system.

In [8]:
nframes = 50
tmax = 100
t_array = linspace(0,tmax,nframes)

mlist = [1,1,1]


solutions = odeint(ode_system, (-2,0,0,-0.4,2,0,0,0.4,3,0,0,-0.01), t_array, args=(mlist,))

x1_array = solutions[:,0]
y1_array = solutions[:,1]

x2_array = solutions[:,4]
y2_array = solutions[:,5]

x3_array = solutions[:,8]
y3_array = solutions[:,9]


plotSomeStuff([x1_array,x2_array,x3_array],[y1_array,y2_array,y3_array], nframes)

Now try out a 4-body system. What can you learn about that?

In [ ]:
nframes = 50
tmax = 100
t_array = linspace(0,tmax,nframes)

mlist = [1,1,1,1]


solutions = odeint(ode_system, (-2,0,0,-0.4,2,0,0,0.4,-5,0,0,-0.01,0,0,0,0.4), t_array, args=(mlist,))

x1_array = solutions[:,0]
y1_array = solutions[:,1]

x2_array = solutions[:,4]
y2_array = solutions[:,5]

x3_array = solutions[:,8]
y3_array = solutions[:,9]

x4_array = solutions[:,12]
y4_array = solutions[:,13]


plotSomeStuff([x1_array,x2_array,x3_array,x4_array],[y1_array,y2_array,y3_array,y4_array], nframes)

/sw/isaac/applications/anaconda3/2021.05/rhel8_gcc10.2.0/anaconda3-2021.05/lib/python3.8/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


**EC(+0.5): Code up an N-body problem using the functions already defined. But in the handling of the initial conditions, masses, the parsing of the solutions, and the handing of the x and y arrays to the plotting function is made dynamic using lists, arrays, dicts, whatever. i.e. build a system that can scale to a large number of particles without blowing up the number of lines of code. Prove that it works by plotting a 5 particle system. If the number of lines you have scales with the number of particles, you're doing it wrong.**